In [ ]:
import requests
import shutil
requests.packages.urllib3.disable_warnings()
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

In [ ]:
token="MDAxNWxvY2F00aW9uIG9uZXpvbmUKMDAzMGlkZW500aWZpZXIgOTIzNDBmZjkyYTI1Y2RjZTlhM2ZmMWIyYTE5MGJjMGEKMDAxYWNpZCB00aW1lIDwgMTU3NTM4MDY00MwowMDJmc2lnbmF00dXJlIGnMtasbKyvaMI84gZo0061QqELeHb1KJBFJulqOmTdBsCg"
host="lapp-xdc01.in2p3.fr"
space="615307e1c80add12526796e5fc197cad"
startRangeFilter="[1467294695, null, null, null, null, null]"
endRangeFilter="[1467294695, null, null, null, null, null]"
resultBoxQuery=widgets.Text( disabled=True)
resultBoxRetrieve=widgets.Text( disabled=True)


In [ ]:
def query(startRangeFilter, endRangeFilter):
    queryUrl = "https://"+host+"/api/v3/oneprovider/spaces/"+space+"/indexes/SimulationIndex/query?"
    queryParam = {'spatial': 'true','start_range': startRangeFilter,'end_range': endRangeFilter}
    queryHeaders = {'X-Auth-Token': token,"Content-type": "application/json"}
    queryResponse = requests.request("GET", queryUrl,  headers=queryHeaders, params=queryParam, verify=False)
    resultBoxQuery.value="number of file return : "+str(len(queryResponse.json()))
    #print (queryResponse.json())
    return (queryResponse.json())


In [ ]:
def retrieveFile(objectID):
    retrieveHeaders = {'X-Auth-Token': token,"Content-type": "application/cdmi-container"}
    response = requests.get('https://lapp-xdc01.in2p3.fr/cdmi/cdmi_objectid/'+objectID, headers=retrieveHeaders, verify=False, stream=True)

    if response.status_code == 200:
        with open(objectID+".simtel.gz", 'wb') as f:
            response.raw.decode_content = True
            shutil.copyfileobj(response.raw, f)
            resultBoxRetrieve.value="copy done"
            

In [ ]:
def retrieveFiles(startRangeFilter,endRangeFilter):
    i=0
    jsonResponse=query(startRangeFilter,endRangeFilter)
    for fileParameter in jsonResponse:
        i=+1
        resultBoxRetrieve.value='Downloading files '+str(i)+'....'
        retrieveFile(fileParameter.get('value'))  


In [7]:
layoutSliders=Layout(width='70%', height='80px')
timeBoolean=widgets.Checkbox(value=False,description='Time')
time=widgets.IntRangeSlider(min=1467250000,max=1467400000,step=100,description=' ', layout=layoutSliders)
simtel_versionBoolean=widgets.Checkbox(value=False,description='Simtel version')
simtel_version=widgets.IntRangeSlider(min=1460000000,max=1467300000,step=100,description=' ', layout=layoutSliders)
run_idBoolean=widgets.Checkbox(value=False,description='Run ID')
run_id=widgets.IntRangeSlider(min=0,max=10000,step=1,description=' ', layout=layoutSliders)
pointing_direction_azBoolean=widgets.Checkbox(value=False,description='Pointing direction alt')
pointing_direction_az=widgets.FloatRangeSlider(min=0,max= 6.3,step=0.00000001,description=' ', layout=layoutSliders)
pointing_direction_altBoolean=widgets.Checkbox(value=False,description='Pointing direction az')
pointing_direction_alt=widgets.FloatRangeSlider(min=0,max= 2,step=0.00000001,description=' ', layout=layoutSliders)
particle_typeBoolean=widgets.Checkbox(value=False,description='Particle type')
particle_type=widgets.Dropdown(options=['0', '1', '2'],description=' ')
buttonQuery = widgets.Button(description="send query")
buttonRetreive = widgets.Button(description="execute retreive")



def updateField(timeBooleanValue, timeValue, simtel_versionBooleanValue, simtel_versionValue, run_idBooleanValue, run_idValue, pointing_direction_azBooleanValue, pointing_direction_azValue, pointing_direction_altBooleanValue, pointing_direction_altValue, particle_typeBooleanValue, particle_typeValue):
    time.disabled=(not timeBooleanValue)
    simtel_version.disabled=(not simtel_versionBooleanValue)
    run_id.disabled=(not run_idBooleanValue)
    pointing_direction_az.disabled=(not pointing_direction_azBooleanValue)
    pointing_direction_alt.disabled=(not pointing_direction_altBooleanValue)
    particle_type.disabled=(not particle_typeBooleanValue)
    return 

def getRangeStart():
    startRangeFilter="["+str(time.value[0] if not time.disabled else "null")+","+str(simtel_version.value[0] if not simtel_version.disabled else "null")+","+str(run_id.value[0] if not run_id.disabled else "null")+","+str(pointing_direction_az.value[0] if not pointing_direction_az.disabled else "null")+","+str(pointing_direction_alt.value[0] if not pointing_direction_alt.disabled else "null")+","+str(particle_type.value if not particle_type.disabled else "null")+"]"
    #print("startRangeFilter"+startRangeFilter)
    return startRangeFilter

def getRangeEnd():
    endRangeFilter="["+str(time.value[1] if not time.disabled else "null")+","+str(simtel_version.value[1] if not simtel_version.disabled else "null")+","+str(run_id.value[1] if not run_id.disabled else "null")+","+str(pointing_direction_az.value[1] if not pointing_direction_az.disabled else "null")+","+str(pointing_direction_alt.value[1] if not pointing_direction_alt.disabled else "null")+","+str(particle_type.value if not particle_type.disabled else "null")+"]"
    #print("endRangeFilter"+endRangeFilter)
    return endRangeFilter

def on_button_clicked_query(b):
    query(getRangeStart(),getRangeEnd())
    
def on_button_clicked_retreive(b):
    retrieveFiles(getRangeStart(),getRangeEnd())
    

    
buttonRetreive.on_click(on_button_clicked_retreive)    
buttonQuery.on_click(on_button_clicked_query)
interact(updateField,timeBooleanValue=timeBoolean, timeValue=time, simtel_versionBooleanValue=simtel_versionBoolean, simtel_versionValue=simtel_version, run_idBooleanValue=run_idBoolean, run_idValue=run_id, pointing_direction_azBooleanValue=pointing_direction_azBoolean, pointing_direction_azValue=pointing_direction_az ,  pointing_direction_altBooleanValue=pointing_direction_altBoolean, pointing_direction_altValue=pointing_direction_alt, particle_typeBooleanValue=particle_typeBoolean, particle_typeValue=particle_type);
display(buttonQuery)
display(resultBoxQuery)
display(buttonRetreive)
display(resultBoxRetrieve)



interactive(children=(Checkbox(value=False, description='Time'), IntRangeSlider(value=(1467287500, 1467362500)…

Button(description='send query', style=ButtonStyle())

Text(value='', disabled=True)

Button(description='execute retreive', style=ButtonStyle())

Text(value='', disabled=True)